# Supabase Vector Store
In this notebook we are going to show how to use [Vecs](https://supabase.github.io/vecs/) to perform vector searches in LlamaIndex

In [ ]:
import logging
import sys

# Uncomment for debugging the local copy of Llamaindex you are working on
# sys.path.append('[dir_to_llama_index_for_testing]')

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, Document, StorageContext
from llama_index.indices.vector_store import GPTVectorStoreIndex
from llama_index.vector_stores import SupabaseVectorStore
import textwrap

### Setup OpenAI
The first step is to configure the OpenAI key. It will be used to created embeddings for the documents loaded into the index

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "[your_openai_api_key]"

### Loading documents
Load the documents stored in the `paul_graham_essay/data` using the SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader('../data/paul_graham/').load_data()
print('Document ID:', documents[0].doc_id, 'Document Hash:', documents[0].doc_hash)

### Create an index backed by Supabase's vector store. 
This will work with all Postgres providers that support pgvector.

In [ ]:
vector_store = SupabaseVectorStore(postgres_connection_string="[your_postres_connection_string]", collection_name="[collection_name]")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)

### Query the index
We can now ask questions using our index.

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")

In [ ]:
print(textwrap.fill(str(response), 100))

In [ ]:
response = query_engine.query("What did the author do growing up?")

In [ ]:
print(textwrap.fill(str(response), 100))

### Appending data
You can also add data to an existing index

In [ ]:
del index

index = GPTVectorStoreIndex.from_documents([Document("The sky is blue")], uri="/tmp/new_dataset")

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")
print(textwrap.fill(str(response), 100))

In [ ]:
index = GPTVectorStoreIndex.from_documents(documents, uri="/tmp/new_dataset")

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")
print(textwrap.fill(str(response), 100))